In [26]:
import subprocess
import re
from pathlib import Path

In [44]:
# === Configuration ===
data_path = "/xustrg0/2024B8049"
run_number = "222753"
mask_path = "/UserData/fperakis/test_data_2025/utilities/empty_mask.npy"
output_path = "/UserData/fperakis/test_data_2025/processed"
poni_file = "/UserData/fperakis/test_data_2025/utilities/geometry_test.poni"
nbins = 150
n_phi = 90
n_chunks = 10  # total number of chunks

In [45]:
# === Submit array job ===
env_vars = ",".join([
    f"DATA_PATH={data_path}",
    f"RUN_NUMBER={run_number}",
    f"MASK_PATH={mask_path}",
    f"OUTPUT_PATH={output_path}",
    f"PONI_FILE={poni_file}",
    f"NBINS={nbins}",
    f"N_PHI={n_phi}",
    f"N_CHUNKS={n_chunks}"
])

submit_cmd = [
    "qsub", "-J", f"0-{n_chunks-1}",
    "-v", env_vars,
    "submit_chunk.pbs"
]

array_result = subprocess.run(submit_cmd, capture_output=True, text=True)

# === Extract array job ID ===
match = re.search(r"(\d+)", array_result.stdout)
if not match:
    raise RuntimeError("Failed to parse array job ID. Output:\n" + array_result.stdout)

array_job_id = match.group(1)
print(f"\nArray job submitted with ID: {array_job_id}")

# === Submit dependent merge job ===

# Define parameters
output_file = Path(output_path) / f"Iq_{run_number}_combined.h5"
pbs_script = "combine_chunks.pbs"

# Submit the PBS job
merge_cmd = [
    "qsub",
    "-v", f"INPUT_DIR={Path(output_path)},RUN_NUMBER={run_number},OUTPUT_FILE={output_file}",
    pbs_script
]

result = subprocess.run(merge_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Output results
if result.returncode == 0:
    merge_job_id = result.stdout.strip()
    print(f"Merge job submitted successfully with job ID: {merge_job_id}")
    #print(f"Check logs: logs/combine_chunks.o{merge_job_id.split('.')[0]}")
else:
    print("Failed to submit merge job.")
    print(result.stderr)



Array job submitted with ID: 8421553
Merge job submitted successfully with job ID: 8421554.fep01


In [50]:
# === Check job status ===
!qstat -u fperakis

In [51]:
!ls /UserData/fperakis/test_data_2025/processed/

Iq_222753_chunk00.h5  Iq_222753_chunk04.h5  Iq_222753_chunk08.h5
Iq_222753_chunk01.h5  Iq_222753_chunk05.h5  Iq_222753_chunk09.h5
Iq_222753_chunk02.h5  Iq_222753_chunk06.h5  Iq_222753_combined.h5
Iq_222753_chunk03.h5  Iq_222753_chunk07.h5


In [52]:
# === Submit dependent merge job ===

# Define parameters
output_file = Path(output_path) / f"Iq_{run_number}_combined.h5"
pbs_script = "combine_chunks.pbs"

# Submit the PBS job
merge_cmd = [
    "qsub",
    "-v", f"INPUT_DIR={Path(output_path)},RUN_NUMBER={run_number},OUTPUT_FILE={output_file}",
    pbs_script
]

result = subprocess.run(merge_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Output results
if result.returncode == 0:
    merge_job_id = result.stdout.strip()
    print(f"Merge job submitted successfully with job ID: {merge_job_id}")
    #print(f"Check logs: logs/combine_chunks.o{merge_job_id.split('.')[0]}")
else:
    print("Failed to submit merge job.")
    print(result.stderr)


Merge job submitted successfully with job ID: 8421557.fep01
